In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pdffff/BSE102_Lecture 3_Chemical basis of life.pdf
/kaggle/input/bsebse/BSE102_Lecture 3_Chemical basis of life.pdf
/kaggle/input/ddddddd/pdf-sample.pdf


In [45]:
!pip install pymupdf
!pip install einops flash_attn
!pip install python-docx


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [46]:
!pip install flash_attn einops

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [47]:
import os
import fitz  
from PIL import Image, ImageDraw, ImageFont
import io
from io import BytesIO
import pytesseract  
import torch 
import requests  
from transformers import Blip2Processor, Blip2ForConditionalGeneration, PegasusForConditionalGeneration, PegasusTokenizer, AutoProcessor, AutoModelForCausalLM 
from docx import Document 
from docx.shared import Inches  
import tempfile  
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import BartForConditionalGeneration, BartTokenizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import BartTokenizer
import numpy as np 
import pandas as pd 
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [49]:
model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large", trust_remote_code=True).to(device)
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large", trust_remote_code=True)

In [50]:
def run_example(image, task_prompt, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input
    inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        num_beams=3
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(generated_text, task=task_prompt, image_size=(image.width, image.height))
    caption = parsed_answer["<MORE_DETAILED_CAPTION>"]
    return caption

In [51]:
def pdf_doc(pdf_document,start,end):
    doc = fitz.open(pdf_document)

    page_text_list = []
    for page_number in range(start-1,end):
        page = doc.load_page(page_number)  
        image_list = page.get_images(full=True)  

        page_text = page.get_text()
        page_text_combined = page_text

        
        for img_index, img in enumerate(image_list):
            xref = img[0] 
            base_image = doc.extract_image(xref)  
            image_bytes = base_image["image"]
            
            image = Image.open(io.BytesIO(image_bytes)).convert('RGB')
            
            prompt = "<MORE_DETAILED_CAPTION>"
            generated_caption = run_example(image, prompt)

            combined_text = f"\n\nImage {img_index + 1}:\nCaption: {generated_caption}\n"

            page_text_combined += combined_text

        page_text_list.append(page_text_combined)
    return page_text_list

In [52]:
def docx_doc(docx_document,start,end):
    doc = Document(docx_document)
    doc_text_list = []
    for para in doc.paragraphs:
        doc_text_list.append(para.text)

        for run in para.runs:
            if run.element.xml.find("pic:blipFill") != -1:
                image = run.element.xpath(".//a:blip")[0]
                image_data = image.get("{http://schemas.openxmlformats.org/officeDocument/2006/relationships}embed")
                image_part = doc.part.related_parts[image_data]
                image_bytes = image_part.blob

                image = Image.open(io.BytesIO(image_bytes)).convert('RGB')
                prompt = "<MORE_DETAILED_CAPTION>"
                generated_caption = run_example(image, prompt)
                combined_text = f"Caption: {generated_caption}\n"
                doc_text_list.append(combined_text)
    return doc_text_list

    



In [53]:
def determine_file_type(file_path):
    _, file_extension = os.path.splitext(file_path)
    if file_extension.lower() == '.docx':
        return 'docx'
    elif file_extension.lower() == '.pdf':
        return 'pdf'
    else:
        return 'unknown'

In [54]:
def get_text(path,start,end):
    text_each_page=[]
    file_type=determine_file_type(path)
    if file_type=='pdf':
        text_each_page=pdf_doc(path,start,end)
    elif file_type=='docx':
        text_each_page=docx_doc(path)
    else :
        print("invalid file type")
    return text_each_page

In [55]:
def preprocess_text(text,start,end):
    text = text.lower()
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)


In [56]:
def process_text_in_sentence_chunks(path, max_chunk_size,start,end):
    text=('\n'.join(page for page in get_text(path,start,end)))
    sentences = text.replace("\n", " ") 
    sentences = nltk.sent_tokenize(sentences)

    current_chunk = ""
    text_list = []
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_chunk_size:
            current_chunk += " " + sentence
        else:
              text_list.append(current_chunk.strip())
              current_chunk = sentence

    if current_chunk:
        text_list.append(current_chunk.strip())
    return text_list

In [57]:
user={
    "path":"/kaggle/input/pdffff/BSE102_Lecture 3_Chemical basis of life.pdf",
    "start":2,
    "end":11,
}

In [58]:
input_text=get_text(user["path"],user["start"],user["end"])


In [59]:
input_text="\n".join(input_text)

In [60]:
len(input_text)

22100

In [61]:
# import torch
# from transformers import LEDTokenizer, LEDForConditionalGeneration

# tokenizer = LEDTokenizer.from_pretrained("hyesunyun/update-summarization-bart-large-longformer")
# model = LEDForConditionalGeneration.from_pretrained("hyesunyun/update-summarization-bart-large-longformer")

# # input =<abs> ETHNOPHARMACOLOGICAL RELEVANCE: Bitter melon (Momordica charantia L.) has been widely used as an traditional medicine treatment for diabetic patients in Asia. In vitro and animal studies suggested its hypoglycemic activity, but limited human studies are available to support its use. AIM OF STUDY: This study was conducted to assess the efficacy and safety of three doses of bitter melon compared with metformin. MATERIALS AND METHODS: This is a 4-week, multicenter, randomized, double-blind, active-control trial. Patients were randomized into 4 groups to receive bitter melon 500 mg/day, 1,000 mg/day, and 2,000 mg/day or metformin 1,000 mg/day. All patients were followed for 4 weeks. RESULTS: There was a significant decline in fructosamine at week 4 of the metformin group (-16.8; 95% CI, -31.2, -2.4 mumol/L) and the bitter melon 2,000 mg/day group (-10.2; 95% CI, -19.1, -1.3 mumol/L). Bitter melon 500 and 1,000 mg/day did not significantly decrease fructosamine levels (-3.5; 95% CI -11.7, 4.6 and -10.3; 95% CI -22.7, 2.2 mumol/L, respectively). CONCLUSIONS: Bitter melon had a modest hypoglycemic effect and significantly reduced fructosamine levels from baseline among patients with type 2 diabetes who received 2,000 mg/day. However, the hypoglycemic effect of bitter melon was less than metformin 1,000 mg/day. <EV> <t> Momordica charantia for type 2 diabetes mellitus. <abs> There is insufficient evidence to recommend momordica charantia for type 2 diabetes mellitus. Further studies are therefore required to address the issues of standardization and the quality control of preparations. For medical nutritional therapy, further observational trials evaluating the effects of momordica charantia are needed before RCTs are established to guide any recommendations in clinical practice."
# input="<abs"+input_text
# inputs_dict = tokenizer(input, padding="max_length", max_length=4096, return_tensors="pt", truncation=True)
# input_ids = inputs_dict.input_ids
# attention_mask = inputs_dict.attention_mask
# global_attention_mask = torch.zeros_like(attention_mask)
# # put global attention on <s> token
# global_attention_mask[:, 0] = 1

# predicted_summary_ids = model.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)
# print(tokenizer.batch_decode(predicted_summary_ids, skip_special_tokens=True))


In [63]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("suriya7/bart-finetuned-text-summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("suriya7/bart-finetuned-text-summarization")

def generate_summary(text):
    inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_new_tokens=4096, do_sample=False)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

text_to_summarize = input_text
summary = generate_summary(text_to_summarize)
print(summary)


An infographic has been created to show the different elements of the human body, as well as information about the effects of certain elements on human health, including the risk of developing goiter, goitre, and goiter (goitre is a swelling of the thyroid gland).
